# Policy Gradients


Policy gradient to metoda uczenia przez wzmacnianie, która polega na **bezpośrednim uczeniu strategii**, czyli funkcji $ \pi_\theta(a|s) $, która określa prawdopodobieństwo wyboru akcji $ a $ w stanie $ s $. Zamiast optymalizować funkcję wartości (jak w Q-learning), policy gradient optymalizuje samą strategię, tak aby maksymalizować długoterminową oczekiwaną nagrodę $ J(\pi_\theta) $.

**Podstawowe kroki policy gradient**:

1. **Próbkowanie doświadczeń**:
- Agent działa w środowisku zgodnie z aktualną strategią, zbierając trajektorie (stany, akcje, nagrody).

2. **Obliczanie gradientu**:
- Funkcja celu (np. REINFORCE) opiera się na policy gradient theorem:
  $$
  \nabla_\theta J(\pi_\theta) = \mathbb{E}[\nabla_\theta \log \pi_\theta(a|s) \cdot R(\tau)]
  $$
  - $ \nabla_\theta \log \pi_\theta(a|s) $: Jak zmienić parametry $ \theta $, aby zwiększyć prawdopodobieństwo akcji $ a $ w stanie $ s $.
  - $ R(\tau) $: Nagroda z całej trajektorii, używana do oceny, jak "dobry" był wybór akcji.

3. **Aktualizacja strategii**:
- Parametry $ \theta $ są aktualizowane w kierunku, który zwiększa prawdopodobieństwo akcji prowadzących do wysokich nagród.




Poniżej znajduje się najprostsza implementacja *policy gradients* na przykładzie środowiska `CartPole-v0`. Osiąga ona zwrot maksymalny 195. [Leaderboard](https://github.com/openai/gym/wiki/Leaderboard#cartpole-v0) wynosi $382$). Poniższa implementacja osiąga ~194, co możemy uznać za rozwiązanie problemu `CartPole`.

In [ ]:
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.optim import Adam
import numpy as np
import gym
from gym.spaces import Discrete, Box

def mlp(sizes, activation=nn.Tanh, output_activation=nn.Identity):
    # Build a feedforward neural network.
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)

def train(env_name='CartPole-v0', hidden_sizes=[32], lr=1e-2,
          epochs=100, batch_size=5000, render=False):

    # make environment, check spaces, get obs / act dims
    env = gym.make(env_name)
    assert isinstance(env.observation_space, Box), \
        "This example only works for envs with continuous state spaces."
    assert isinstance(env.action_space, Discrete), \
        "This example only works for envs with discrete action spaces."

    obs_dim = env.observation_space.shape[0]
    n_acts = env.action_space.n

    # make core of policy network
    logits_net = mlp(sizes=[obs_dim]+hidden_sizes+[n_acts])

    # make function to compute action distribution
    def get_policy(obs):
        logits = logits_net(obs)
        return Categorical(logits=logits)

    # make action selection function (outputs int actions, sampled from policy)
    def get_action(obs):
        return get_policy(obs).sample().item()

    # make loss function whose gradient, for the right data, is policy gradient
    def compute_loss(obs, act, weights):
        logp = get_policy(obs).log_prob(act)
        return -(logp * weights).mean()

    # make optimizer
    optimizer = Adam(logits_net.parameters(), lr=lr)

    # for training policy
    def train_one_epoch():
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for R(tau) weighting in policy gradient
        batch_rets = []         # for measuring episode returns
        batch_lens = []         # for measuring episode lengths

        # reset episode-specific variables
        obs = env.reset()       # first obs comes from starting distribution
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep

        # render first episode of each epoch
        finished_rendering_this_epoch = False

        # collect experience by acting in the environment with current policy
        while True:

            # rendering
            if (not finished_rendering_this_epoch) and render:
                env.render()

            # save obs
            batch_obs.append(obs.copy())

            # act in the environment
            act = get_action(torch.as_tensor(obs, dtype=torch.float32))
            obs, rew, done, _ = env.step(act)

            # save action, reward
            batch_acts.append(act)
            ep_rews.append(rew)

            if done:
                # if episode is over, record info about episode
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)
                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)

                # the weight for each logprob(a|s) is R(tau)
                batch_weights += [ep_ret] * ep_len

                # reset episode-specific variables
                obs, done, ep_rews = env.reset(), False, []

                # won't render again this epoch
                finished_rendering_this_epoch = True

                # end experience loop if we have enough of it
                if len(batch_obs) > batch_size:
                    break

        # take a single policy gradient update step
        optimizer.zero_grad()
        batch_loss = compute_loss(obs=torch.as_tensor(batch_obs, dtype=torch.float32),
                                  act=torch.as_tensor(batch_acts, dtype=torch.int32),
                                  weights=torch.as_tensor(batch_weights, dtype=torch.float32)
                                  )
        batch_loss.backward()
        optimizer.step()
        return batch_loss, batch_rets, batch_lens

    # training loop
    for i in range(epochs):
        batch_loss, batch_rets, batch_lens = train_one_epoch()
        print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
                (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))

if __name__ == '__main__':
    print('\nUsing simplest formulation of policy gradient.\n')
    train(env_name='CartPole-v0', render=True, lr=1e-2)


Using simplest formulation of policy gradient.



/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


epoch:   0 	 loss: 15.526 	 return: 18.281 	 ep_len: 18.281
epoch:   1 	 loss: 18.902 	 return: 21.496 	 ep_len: 21.496
epoch:   2 	 loss: 21.356 	 return: 23.952 	 ep_len: 23.952
epoch:   3 	 loss: 24.615 	 return: 27.388 	 ep_len: 27.388
epoch:   4 	 loss: 29.083 	 return: 33.007 	 ep_len: 33.007
epoch:   5 	 loss: 29.323 	 return: 34.397 	 ep_len: 34.397
epoch:   6 	 loss: 29.674 	 return: 36.326 	 ep_len: 36.326
epoch:   7 	 loss: 31.892 	 return: 38.313 	 ep_len: 38.313
epoch:   8 	 loss: 28.470 	 return: 37.228 	 ep_len: 37.228
epoch:   9 	 loss: 34.139 	 return: 42.863 	 ep_len: 42.863
epoch:  10 	 loss: 31.716 	 return: 42.390 	 ep_len: 42.390
epoch:  11 	 loss: 31.855 	 return: 44.652 	 ep_len: 44.652
epoch:  12 	 loss: 36.741 	 return: 50.050 	 ep_len: 50.050
epoch:  13 	 loss: 38.042 	 return: 51.670 	 ep_len: 51.670
epoch:  14 	 loss: 33.503 	 return: 48.163 	 ep_len: 48.163
epoch:  15 	 loss: 40.256 	 return: 57.330 	 ep_len: 57.330
epoch:  16 	 loss: 34.951 	 return: 52.3

## Implementacja strategii

### Sieć neuronowa strategii

```python
# make core of policy network
logits_net = mlp(sizes=[obs_dim]+hidden_sizes+[n_acts])

# make function to compute action distribution
def get_policy(obs):
    logits = logits_net(obs)
    return Categorical(logits=logits)

# make action selection function (outputs int actions, sampled from policy)
def get_action(obs):
    return get_policy(obs).sample().item()
```

Ten blok buduje najprostrzą sieć neuronową typu perceptron dla strategii *kategorycznej* (czyli takiej, która ma dyskretną liczbę akcji).  
  
Wyjściem z `logits_net` są logity, czyli logarytmy prawdopodobieństw akcji. Funkcja `get_action` próbkuje akcję na podstawie prawdopodobieństw obliczonych z logitów.  

```
Sieć zwraca logity (log prawdopodobieństwa), ponieważ:

- są stabilniejsze numerycznie.
- ułatwiają obliczenia z logarytmem prawdopodobieństw.
- mogą być łatwo przekształcone w prawdopodobieństwa za pomocą Softmax.
- dają sieci więcej swobody w nauce.

To standardowe podejście w modelach, które pracują z kategoriami i rozkładami prawdopodobieństwa.
```

(**Uwaga**: ta konkretna funkcja `get_action` zakłada, że zostanie dostarczona tylko jedna obserwacja `obs`, a zatem tylko jedna liczba całkowita reprezentująca akcję. Dlatego używa `.item()`, który pobiera zawartość tensora zawierającego tylko jeden element).

Duża część pracy w tym przykładzie jest wykonywana przez obiekt `Categorical`.  
Jest to obiekt z PyTorch, który obejmuje szereg funkcji matematycznych związanych z rozkładami prawdopodobieństwa.  
W szczególności zawiera metodę do próbkowania z rozkładu (którą używamy na linii 40) oraz metodę do obliczania log-prawdopodobieństw dla danych próbek (które użyjemy później).  
Ponieważ rozkłady w PyTorch są bardzo przydatne dla RL, sprawdź ich [dokumentację](https://pytorch.org/docs/stable/distributions.html), aby lepiej zrozumieć, jak działają.







#### Struktura sieci

Sieć neuronowa zdefiniowana w tym kodzie to **perceptron wielowarstwowy (MLP)** zbudowany za pomocą funkcji `mlp`:

1. **Warstwa wejściowa**:
   - Rozmiar: Liczba cech w przestrzeni obserwacji środowiska (`obs_dim`).
   - Przykład (dla `CartPole-v0`): Rozmiar wejściowy = 4 (pozycja wózka, prędkość wózka, kąt kija, prędkość kątowa kija).
   ```python
   obs = [0.0169022, -0.00525023, 0.0194337, 0.0132078]
   ```

2. **Warstwy ukryte**:
   - Konfigurowane za pomocą parametru `hidden_sizes`. W tym przykładzie jest jedna warstwa ukryta z 32 neuronami, z funkcją aktywacji Tanh.

3. **Warstwa wyjściowa**:
   - Rozmiar: Liczba dostępnych akcji w środowisku (`n_acts`).
   - Przykład (dla `CartPole-v0`): Rozmiar wyjściowy = 2 (lewo lub prawo).
   ```python
   logits = [1.5, -0.8]
   ```
   Są to nieznormalizowane wartości dla każdej akcji (lewo i prawo). Normalizacja jest dokonywana w [Categorical](https://pytorch.org/docs/stable/distributions.html#categorical) przy pomocy funkcji [Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html). Inaczej mówiąc, Logity są przekształcane w prawdopodobieństwa za pomocą funkcji softmax.


### Funkcja straty

**Celem jest dostosowanie parametrów strategii $ \theta $ w kierunku, który zwiększa prawdopodobieństwo działań prowadzących do wyższych nagród.**

```python
# make loss function whose gradient, for the right data, is policy gradient
def compute_loss(obs, act, weights):
    logp = get_policy(obs).log_prob(act)
    return -(logp * weights).mean()
```


Wejściem do funkcji "straty" (właściwie celu) jest "batch" trajektorii, na który składają się obserwacje, akcje oraz suma nagród uzyskana dla danej trajektorii. Przykład (pojedynczego elementu z batcha):
```
obs = [[0.5, 1.2], [0.3, -0.7]]
act = [0, 1]
weights = [50, 50]
```

Każda logarytmiczna wartość prawdopodobieństwa jest skalowana odpowiednią wagą (w najprostszej formie algorytmu gradientu strategii, stosowanej tutaj, ta sama wartość zwrotu jest przypisywana wszystkim działaniom w ramach jednej trajektorii).  
To **ważenie** zapewnia aktualizację gradientu w celu zwiększania prawdopodobieństwa działań, które przyniosły wyższe nagrody.  
**Średnia** zapewnia, że funkcja straty reprezentuje średnią wydajność dla wszystkich trajektorii w batchu.  
Dzięki **odwróceniu znaku** funkcji straty, optymalizator zwiększa prawdopodobieństwo działań przynoszących większe nagrody (większość optymalizatorów, np. Adam, minimalizuje funkcję straty, ale w tym przypadku chcemy maksymalizować cel).

**Intuicja:**  
- Działania z wyższymi zwrotami (nagrodami) mają większe wagi, więc ich logarytmiczne prawdopodobieństwa są bardziej zwiększane.  
- Działania z niższymi zwrotami mają mniejsze wagi (lub ujemne wagi w przypadku kar), więc ich logarytmiczne prawdopodobieństwa są zmniejszane.

## Pętla trenująca

1. **Zbieranie doświadczeń**:
   - Agent działa w środowisku, wykonując akcje oparte na bieżącej polityce (`get_action()`).
   - Obserwacje, wykonane akcje i otrzymane nagrody są zapisywane.

2. **Koniec epizodu**:
   - Po zakończeniu epizodu (sygnał `done`) sumowana jest całkowita nagroda $ R(\tau) $ oraz długość epizodu.
   - Wagi (`weights`) są ustawiane jako powtórzona wartość $ R(\tau) $ dla każdego kroku epizodu.

3. **Warunek zakończenia**:
   - Jeśli liczba zebranych obserwacji przekracza `batch_size`, kończy się zbieranie danych w tej epoce.

4. **Aktualizacja polityki**:
   - Funkcja straty (`compute_loss`) oblicza gradient na podstawie zebranych danych.
   - Optymalizator aktualizuje parametry sieci neuronowej na podstawie obliczonego gradientu.


Pętla ta powtarza się przez określoną liczbę epok (`epochs`), zbierając doświadczenia, aktualizując politykę i poprawiając działania agenta w środowisku.

# Zadanie

1. Pokaż wykres **zwrotu w kolejnych epokach**: ilustruje poprawę w sumie nagród zdobytych w trajektorii, co jest bezpośrednią miarą wydajności agenta.
2. Zaimplementuj **baseline**. W metodach policy gradient baseline to element odejmowany od nagrody (lub zwrotu, $R(\tau)$), który ma na celu zmniejszenie wariancji oszacowań gradientu. Dzięki temu proces uczenia staje się bardziej stabilny i szybszy.

Prostym i skutecznym **baseline** jest średnia zwrotów zebranych w bieżym batchu. Zamiast bezpośrednio używać zwrotów $R(\tau)$ w *loss-function*, odejmujemy od nich ich średnią $mean(R(\tau))$.


1. **Oblicz baseline**: Wyznacz średnią wszystkich zwrotów w batchu:
   $$
   b = \frac{1}{N} \sum_{i=1}^N R_i
   $$

   gdzie $N$ to liczba epizodów w batchu.

2. **Odejmij baseline**: Dostosuj wagi używane w obliczeniach gradientu:
   $$
   \text{Adjusted weight} = R_i - b
   $$

3. **Zaktualizuj funkcję straty**: Zmodyfikuj funkcję `compute_loss`, aby uwzględnić baseline.

Porównaj wykresy *zwrotu w kolejnych epokach* dla algorytmu z i bez *baseline*

# Materiały

1. [Deep RL Bootcamp Lecture 4A: Policy Gradients](https://www.youtube.com/watch?v=S_gwYj1Q-44)
2. [Deep RL Bootcamp Lecture 4B Policy Gradients Revisited
](https://www.youtube.com/watch?v=tqrcjHuNdmQ)
3. [SpinningUp OpenAI Intro to PO](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html)
4. [The spelled-out intro to neural networks and backpropagation: building micrograd](https://www.youtube.com/watch?v=VMj-3S1tku0&t)